In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import RobustScaler, StandardScaler
import datetime 

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/Shareddrives/cj공모전/2021년 미래기술 챌린지_211005/1. 이커머스 FC주문 데이터/

/content/gdrive/Shareddrives/cj공모전/2021년 미래기술 챌린지_211005/1. 이커머스 FC주문 데이터


In [ ]:
%ls

cluster_label.pkl             fc주문_6월.csv        전체.pkl
fc주문_통합_수정_1014ver.pkl  fc주문_통합_수정.csv  모델링파일.pkl
fc주문_3월.csv                fc주문_통합.csv       시계열temp1.csv
fc주문_4월.csv                fc주문_통합_수정.pkl  시계열temp1.pkl
fc주문_5월.csv                fc주문_통합_ver2.csv


In [ ]:
final_df = pd.read_pickle('fc주문_통합_수정.pkl')

날짜 지역 가격 INDEX 쇼핑몰 코드별로 CSV파일 생성

In [ ]:
final_df.head()

,CORP_ID,REF_ORD_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_CD,ITEM_QTY,ITEM_AMT,REF_ITEM_SEQ,DLVPREARRBRANCD,DLVPREARREMPNICKNM,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2,개별가격,평균가격,metric,cluster
0,KX007,2021030365922551,7,2021-03-03,035510,90001302,22400.00000,134265392,1,14900.00000,2021030323885041.0,7484,C06,5Z73,1,3.0,세종특별자치시,nan,서울,서대문구,14900.00000,14900.00000,134265392,8.0
1,KX007,2021030365922951,7,2021-03-03,035521,90001441,20900.00000,4610261043,1,20900.00000,2021030323885721.0,6427,M09,6R67,4,50.0,경기도,광주시,경남,김해시,20900.00000,20529.68342,4610261043,0.0
2,KX007,2021030365924361,7,2021-03-03,035513,90001441,37900.00000,4936599538,1,37900.00000,2021030323887981.0,6158,L32,9Y25,0,53.0,경기도,광주시,경남,거제시,37900.00000,27175.94846,4936599538,8.0
3,KX007,2021030365928921,7,2021-03-03,035608,90001542,69800.00000,3858391637,1,69800.00000,2021030323895281.0,8909,D21,1N65,1,6.0,경기도,광주시,서울,서초구,69800.00000,102245.81445,3858391637,8.0
4,KX007,2021030365930451,7,2021-03-03,035551,90001341,73700.00000,4631307460,1,39800.00000,2021030323897651.0,2777,R15,3D84,1,10.0,경기도,광주시,경기,김포시,39800.00000,39800.00000,4631307460,8.0


In [ ]:
#클러스터 빈거 10으로 채움
final_df['cluster'].replace({'nan':'10'},inplace = True)

In [ ]:
#평균가격 구하기(단순 나누기), 날짜별로 달라지는걸 보여주고 싶음
final_df['ITEM_PRICE'] = final_df.apply(lambda x: x['ITEM_AMT']/x['ITEM_QTY'], axis = 1)

In [ ]:
final_df.SHPR_CD.unique()

array(['90001302', '90001441', '90001542', '90001341', '90001541',
       '90001443', '90001381', '90001521', '90001582', '90001602',
       '90001662', '90001622', '90001682', '90001683', '90001702',
       '90001703', '90001705', '90001704', '90001664', '90001768',
       '90001765', '90001776', '90001774', '90001842'], dtype=object)

In [ ]:
final_df.columns

Index(['CORP_ID', 'REF_ORD_NO', 'BKG_TYP', 'BKG_DATE', 'BKG_TIME', 'SHPR_CD',
       'INV_AMT', 'ITEM_CD', 'ITEM_QTY', 'ITEM_AMT', 'REF_ITEM_SEQ',
       'DLVPREARRBRANCD', 'DLVPREARREMPNICKNM', 'DLVCLSFCD', 'DLVSUBCLSFCD',
       'POST_ZONE', 'SHPR_ADDR_1', 'SHPR_ADDR_2', 'CNEE_ADDR_1', 'CNEE_ADDR_2',
       '개별가격', '평균가격', 'metric', 'cluster', 'ITEM_PRICE'],
      dtype='object')

In [ ]:
%cd /content/gdrive/Shareddrives/cj공모전/외부 데이터/

/content/gdrive/Shareddrives/cj공모전/외부 데이터


In [ ]:
%ls

'외부 데이터 전처리'/
 기온/
 코로나/
 강수량/
 공시지가/
 국가공휴일_03_06.csv
 공시지가_시군구_2021_01.zip
 202103_202106_연령별인구현황_월간.csv
'202103_202106_주민등록인구기타현황(세대원수별 세대수)_households.csv'
'네이버 쇼핑 트렌드_통합.csv'
 holiday_cleaned.csv
 rain.csv
 rain_ed.csv
 rain.gsheet
 temperature.csv
 temperature_ed.csv
 temperature.gsheet
'네이버 검색어트렌드.xlsx'


In [ ]:
hc = pd.read_csv("holiday_cleaned.csv")

In [ ]:
hc.dtypes

날짜      object
설명      object
요일       int64
휴일여부     int64
dtype: object

In [ ]:
hc.head()

,날짜,설명,요일,휴일여부
0,2018-01-01,새해,0,1
1,2018-01-02,해당없음,1,0
2,2018-01-03,해당없음,2,0
3,2018-01-04,해당없음,3,0
4,2018-01-05,해당없음,4,0


In [ ]:
hc['날짜'] = hc['날짜'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))

In [ ]:
hc = hc[(hc['날짜']>='2021-03-01') &(hc['날짜']<='2021-06-30')]

In [ ]:
hc.head()

,날짜,설명,요일,휴일여부
1155,2021-03-01,삼일절,0,1
1156,2021-03-02,해당없음,1,0
1157,2021-03-03,해당없음,2,0
1158,2021-03-04,해당없음,3,0
1159,2021-03-05,해당없음,4,0


In [ ]:
hc = hc[['날짜','요일','휴일여부']]

In [ ]:
hc.columns = ['BKG_DATE', '요일','휴일여부']

In [ ]:
lll = ['90001702', '90001705']

In [ ]:
#shpr분리때매 비어있는 컬럼 확인하기 위해 컬럼 생성
calt = list(final_df.CNEE_ADDR_1.unique())
tlt = calt

In [ ]:
##누락 되어있는 애들로만 df생성 >  아래서 cdf에 할당하는것
def make_df(temp):
  collist = []
  for i in tlt:
    if i in temp.columns:
      pass
    else : 
      collist.append(i)
  cdf = pd.DataFrame(columns = collist)
  return cdf

In [ ]:
temp = pd.DataFrame(final_df.ITEM_PRICE.unique(), columns=['ITEM_PRICE'])
fig = px.box(temp, y="ITEM_PRICE")
fig.show()

In [ ]:
temp.max()

ITEM_PRICE   1170900.00000
dtype: float64

In [ ]:
def preprocessing(df, hc):
  date = pd.DataFrame(pd.period_range('2021-03-01', '2021-06-30'))
  date.columns = ['BKG_DATE']
  date['BKG_DATE'] = date['BKG_DATE'].apply(lambda x: x.to_timestamp())
  for i in lll:
    #temp_df : df를 shpr_cd별로 끊어 쓰기
    temp_df = df[df['SHPR_CD']==i]
    #output dataframe은 일별로 1개의 컬럼만 존재해야함
    tmp =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
    tmp1 =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
    tmp2 =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
    tmp3 =temp_df.groupby(['BKG_DATE'])['ITEM_QTY'].sum().reset_index()
    #일별 group by한거를 0301-0630사이 날짜 dataframe에 merge
    #매출이 없는 날에는 fillna(0)
    tmp = pd.merge(date, tmp, on = 'BKG_DATE', how='left')
    tmp.fillna(0, inplace = True)
  
    #휴일 merge
    tmp = pd.merge(tmp, hc, how = 'left', on = 'BKG_DATE')

    #하루전, 2일전, 3일전 판매개수
    tmp1['BKG_DATE']=tmp1['BKG_DATE']+datetime.timedelta(1)
    tmp2['BKG_DATE']=tmp2['BKG_DATE']+datetime.timedelta(2)
    tmp3['BKG_DATE']=tmp3['BKG_DATE']+datetime.timedelta(3)
    tmp1=tmp1.rename(columns={'ITEM_QTY':'DAY_1'})
    tmp2=tmp2.rename(columns={'ITEM_QTY':'DAY_2'})
    tmp3=tmp3.rename(columns={'ITEM_QTY':'DAY_3'})
    DAY_df = pd.merge(tmp, tmp1, how = 'left', on=['BKG_DATE'])
    DAY_df = pd.merge(DAY_df, tmp2, how = 'left', on=['BKG_DATE'])
    DAY_df = pd.merge(DAY_df, tmp3, how = 'left', on=['BKG_DATE'])
    #첫날개수는 0으로
    DAY_df=DAY_df.fillna(0)
    #주별 물품 개수 합계
    DAY_df['week'] = DAY_df['BKG_DATE'].apply(lambda x: x.isocalendar()[1])
    DAY_df['week'] = DAY_df['week'].apply(lambda x: x-8)
    dfn= DAY_df.groupby(['week'])['ITEM_QTY'].sum().reset_index()
    dfn=dfn.rename(columns={'ITEM_QTY':'WEEK_AMT'})
    last_df = pd.merge(DAY_df, dfn, how = 'left', on=['week'])
    

    #일별 평균 가격
    df3=temp_df.groupby(['BKG_DATE'])['ITEM_PRICE'].mean().reset_index()
    df3=df3.rename(columns={'ITEM_PRICE':'MEAN_PRICE'})
    last_df = pd.merge(last_df, df3, how = 'left', on = ['BKG_DATE'])

    #지역별 판매량 group by 후 pivot table
    cneedf = temp_df.groupby(['BKG_DATE','CNEE_ADDR_1'])['ITEM_QTY'].sum().reset_index()
    cneedf = cneedf.pivot(index = 'BKG_DATE', columns = 'CNEE_ADDR_1', values = 'ITEM_QTY')
    last_df = pd.merge(last_df, cneedf, how = 'left', on = 'BKG_DATE')
    last_df.fillna(0, inplace = True)


    cdf = make_df(last_df)
    final = pd.concat([last_df, cdf], axis=1)
    final.fillna(0, inplace=True)
    final[['BKG_DATE', 'ITEM_QTY', '요일', '휴일여부', 'DAY_1', 'DAY_2', 'DAY_3', 'week', 'WEEK_AMT', 'MEAN_PRICE',
       '강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천',
       '전남', '전북', '제주', '충남', '충북']]



    final.to_pickle("/content/gdrive/Shareddrives/cj공모전/최종코드/모델링/shpr_max/item_cd_"+i+".pkl")



In [ ]:
item_fix = final_df[final_df['ITEM_CD']=='8809544011655']
preprocessing(item_fix, hc)